# Practice Session 03: Networks from text

*Introduction to Network Science* (2025/26), UPF

Author: <font color="blue">Alejandro Duarte Whitehair DeZerne</font>

E-mail: <font color="blue">alejandroduarte.whitehair01@estudiant.upf.edu</font>

Date: <font color="blue">20/10/2025</font>

# 1. Create the directed mention network

In [1]:
import io
import json
import gzip
import csv
import re

In [2]:
# Input file
COMPRESSED_INPUT_FILENAME = "data/CovidLockdownCatalonia.json.gz"   # You may need to change this filepath depending on where you've saved the data file

# These are the output files, leave as-is
OUTPUT_ALL_EDGES_FILENAME = "CovidLockdownCatalonia.csv"
OUTPUT_FILTERED_EDGES_FILENAME = "CovidLockdownCatalonia-min-weight-filtered.csv"
OUTPUT_CO_MENTIONS_FILENAME = "CovidLockdownCatalonia-co-mentions.csv"

## 1.1. Extract mentions

In [3]:
# Leave this code as-is

def extract_mentions(text):
    return re.findall("@([a-zA-Z0-9_]{5,20})", text)

print(extract_mentions("RT @elnacionalcat: check this post by @en_blau"))

['elnacionalcat', 'en_blau']


## 1.2. Count mentions


In [4]:
mentions_counter = {}

with gzip.open(COMPRESSED_INPUT_FILENAME, "rt", encoding="utf-8") as input_file:
    for line in input_file:
        tweet = json.loads(line)
    
        author = tweet["user"]["screen_name"]
        text = tweet["full_text"]

        mentions = extract_mentions(text)
        for mention in mentions:
            key = (author, mention)
            mentions_counter[key] = mentions_counter.get(key, 0) + 1

In [5]:
print(mentions_counter[('joanmariapique', 'catalangov')])

9


In [6]:
pairs = []
already_checked = []

for (u, v) in mentions_counter.keys():
    total = mentions_counter.get((u, v), 0) + mentions_counter.get((v, u), 0)
    if total > 8 and ((u, v) not in already_checked and (v, u) not in already_checked):
        already_checked.append((u, v))
        already_checked.append((v, u))
        pairs.append((u, v))

for u, v in pairs:
    total_mentions = mentions_counter.get((u, v), 0) + mentions_counter.get((v, u), 0)
    print("Accounts", u, "and", v, "mention each other", total_mentions, "times")

Accounts wualaswold1 and updayESP mention each other 9 times
Accounts emocionycambio and DrTedros mention each other 16 times
Accounts FXstreetNews and HareshMenghani mention each other 11 times
Accounts MargaXrepublica and MargaXrepublica mention each other 12 times
Accounts Txesnut1 and DrEricDing mention each other 11 times
Accounts joanmariapique and catalangov mention each other 9 times


## 1.3 Save network

Write a file `OUTPUT_ALL_EDGES_FILENAME` with ***all*** the edges in this graph `(Source, Target, Weight)` as a tab-separated file, and `OUTPUT_FILTERED_EDGES_FILENAME` with edges of weight larger or equal to 3.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [7]:
# Leave this code as-is

lines_written = 0

with io.open(OUTPUT_ALL_EDGES_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key in mentions_counter:
        author = key[0]
        mention = key[1]
        weight = mentions_counter[key]
        writer.writerow([author, mention, weight])
        lines_written += 1

print(f"Wrote {lines_written} lines to file {OUTPUT_ALL_EDGES_FILENAME}")

Wrote 34040 lines to file CovidLockdownCatalonia.csv


In [8]:
lines_written = 0

with io.open(OUTPUT_FILTERED_EDGES_FILENAME, "w") as output_file:
    writer = csv.writer(output_file, delimiter='\t', quotechar='"')
    writer.writerow(["Source", "Target", "Weight"])
    for key, weight in mentions_counter.items():
        if weight >= 3:
            author, mention = key
            writer.writerow([author, mention, weight])
            lines_written += 1

print(f"Wrote {lines_written} lines to file {OUTPUT_FILTERED_EDGES_FILENAME}")


Wrote 255 lines to file CovidLockdownCatalonia-min-weight-filtered.csv


# 2. Visualize the directed mention network


First, create a network from the filtered edges by reading the file created in the previous section. 

NetworkX basically involves creating and manipulating different [`Graph` classes](https://networkx.org/documentation/stable/reference/classes/index.html) that hold [nodes](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.nodes.html), [edges](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.edges.html), and node/edge [attributes](https://networkx.org/documentation/stable/reference/functions.html#attributes).

For each line in the file that corresponds to an edge, add that source-target edge (preserving the direction of mentions) with its respective weight (number of mentions) to a network `g`. Since our network of mentions is directed, make sure to use the `nx.DiGraph()` class (stands for "**Di**rected **Graph**").

*Tip: You can use either the `csv.reader()` or `csv.DictReader()` function. They differ in the kind of Python objects they parse entries into, both can be useful!*

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [9]:
# Import required packages

import networkx as nx
import matplotlib.pyplot as plt

In [11]:
#create the directed, weighted mentions network g
g = nx.DiGraph()

g.add_edge()

with io.open(OUTPUT_FILTERED_EDGES_FILENAME, "w") as input_file:
    writer = csv.reader(input_file, delimiter='\t', quotechar='"')


TypeError: DiGraph.add_edge() missing 2 required positional arguments: 'u_of_edge' and 'v_of_edge'

To access the edge attributes (e.g. `weight`) in a `networkx` graph, you can use the [`nx.get_edge_attributes()`](https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.get_edge_attributes.html) function, passing it the attribute you want to obtain. For example, below we display a few of the edges with their weights.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Leave this code as-is

list(nx.get_edge_attributes(g, 'weight').items())[:10]

## 2.1. Extract the largest connected component using NetworkX

Now that we have the network of (filtered) mentions `g`, extract its largest connected component (`lcc`).

For this, check out the [`nx.weakly_connected_components()`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.weakly_connected_components.html#networkx.algorithms.components.weakly_connected_components) function.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Recall that for directed networks, we have two different notions of connectedness: *strongly* and *weakly* connected.

We can check if a graph is connected by using:

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Leave this code as-is

print(f"The network is weakly connected: {nx.is_weakly_connected(g)}")
print(f"The network is strongly connected: {nx.is_strongly_connected(g)}")

<font size="+1" color="red">
Replace this cell with your code to extract from <code>g</code> the largest (weakly) connected component into a subgraph <code>lcc</code>.
</font>

To validate, we can check the connectedness of this subgraph.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Leave this code as-is

print(f"The network is weakly connected: {nx.is_weakly_connected(lcc)}")
print(f"The network is strongly connected: {nx.is_strongly_connected(lcc)}")

## 2.2. Visualize the largest connected component


Create a plot with the largest connected component (`lcc`) using a spring (force-directed) layout.

To make the visualization more intuitive, you should take into account the betweenness of edges as an attribute to the layout algorithm and edge styles, as described below.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

First, calculate the betweenness of all edges in the `lcc` and add it as an attribute `betweenness` to the edges (like `weight` is currently). Check the `networkx` documentation for a function that you can use to calculate the edge betweenness centrality.

Since we have a small, weakly connected network, and we will use the betweenness for visualization purposes only, calculate it on the underirected version of the network. You can use `lcc.to_undirected()`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to add the edge betweenness as an attribute to the edges in `lcc`.</font>

The below code outputs the top 10 edges by betweenness.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Leave this code as-is

top_edges = sorted(lcc.edges(data=True), key=lambda x: x[2]['betweenness'], reverse=True)[:10]
for edge in top_edges:
    print(f"Edge from @{edge[0]} to @{edge[1]} has betweenness {edge[2]['betweenness']:.3f}")

Now we will plot the largest connected component using `networkx` and style it.

---

***Quick NetworkX visualization explainer***

With the `networkx` [drawing](https://networkx.org/documentation/stable/reference/drawing.html) utilities, we can plot networks (as `matplotlib` Figures) and customize node and edge styles any way we like.

As we saw with Cytoscape, the two basic elements of a network visualization are the **layout** and the element (node, edge) **styles**.

The basic visualization function in `networkx` is [`nx.draw()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw.html), which has the following signature:

```python
    nx.draw(g, pos=node_layout, **style_keywords)
```

Very briefly:
* We specify the graph we want to draw as `g`.
* The `pos` parameter takes as argument a dictionary mapping nodes in `g` to positions (coordinates in 2D space). 
    * There are [many functions](https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout) in `networkx` for generating different layouts (i.e. node positions). Below we give an example using [`nx.spring_layout()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html).
* In `style_keywords` we pass the function a set of keyword arguments (either as a dictionary, or one-by-one) that affect the appearance of nodes, edges, and labels. 
    * For the basic set of parameters we can pass, and their formats, see the documentation entry of [`nx.draw_networkx()`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html).

There are basically two approaches for plotting with `networkx`:

* Using the basic `nx.draw()` function, and passing the styles as arguments to that function; or
* Using separate calls to `draw_networkx_nodes()`, `draw_networkx_edges()` and `draw_networkx_labels()` to draw the respective elements separately.

The former approch is simpler, and is recommended at the beginning. The latter approach allows for a few more customizations and is more useful when dealing with large networks where the drawing may take a long time.

Note that under the hood `nx.draw()` uses the `draw_networkx_nodes()`, etc., functions, and it is those lower-level functions that define the names and permitted values of the styling parameters. You should check their documentation for reference.

For more information, see the NetworkX [Drawing](https://networkx.org/documentation/stable/reference/drawing.html) documentation.

*Tip: In the future, if you want to step up your network visualization game, check out the [`netgraph`](https://github.com/paulbrodersen/netgraph) package!*

---

Generating good layouts to plot networkx graphs can be tricky. For this first example, you are given the code to generate a suitable layout for this network using an algorithm of the (familiar) force-directed layout type.

Later on, you will need to generate layouts yourself, so take some time to read the code below and check out what the parameters are for [`nx.spring_layout`](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html).

Note in particular how the `weight` parameter works. (This is the reson why we added the betweenness as an attribute to the network.)

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [ ]:
# Leave this code as-is (for now)

pos = nx.spring_layout(     # The function returns a dictionary with node positions (2D coordinates)...
    lcc,                    # ...for nodes in the largest connected component...
    weight='betweenness',   # ...using edge betweenness as an attractive force...
    method='energy',        # ...using the energy minimization method (see documentation for details)...
    k=0.1,                  # ...with a custom setting for the optimal distance between nodes.
    seed=42          # We also set the RNG seed to make the layout reproducible.
)

To style the nodes and edges in the LCC, we need to pass the relevant attributes as numerical values directly to the drawing function. See the `nx.draw()` documentation for details on the keyword arguments.

Style **nodes** in the largest connected component with:

* Node size proportional to their in-degree
* Node labels whose font size is proportional to their in-degree
<!-- * Node color proportional to in-degree (white=small, blue=large) — Kalo: CAN ADD THIS IF THE PRACTICE SEEMS TO EASY -->

Style **edges** with:

* Edge width proportional to weight
* Arrows at the end of each edge

You can include any other styling that you want — as long as the visualization is *readable* and *informative*.

Save the network as an image using `plt.savefig()` with filename `mentions-largest-cc.png`. (The `networkx` drawing function use the `matplotlib` backend by default, so we can manipulate everything as a `Figure` object.)

*Tip: You do not need to see every detail of the network in this notebook, but make sure the network is big enough and the image is saved with a decent quality.*

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to draw the `lcc`, with the layout given above and the required styling.</font>

<font size="+1" color="red">

Replace this cell with a basic analysis of the network structure, using `networkx` functions (see [Algorithms](https://networkx.org/documentation/stable/reference/algorithms/index.html) and [Graph Functions](https://networkx.org/documentation/stable/reference/functions.html#graph)).

* What is the diameter of the largest connected component, disregarding edge direction?
* What is the size of the largest connected component, both as a number of nodes and as a percentage of the nodes in the graph?
* What is the size of the second largest connected component?

</font>

<font size="+1" color="red">Replace this cell with a brief commentary on what you see in this graph, regarding its structure.</font>


<font size="+1" color="red">
Go back to the code above:

1. Remove the `seed` parameter in the call to `spring_layout()`, re-run the code to calculate `pos`, and visualize the network again.
2. Look at the resulting layout. Do this a few times. 

Comment briefly on what you notice. What does the `seed` parameter do? 
</font>


## 2.3. Examine degree distributions


Analyze the in- and out-degree distributions of nodes in the **largest connected component**:

* Plot both distributions.
* Plot a **scatterplot** of the in and out degree of nodes.

*Important: Make sure the plots are readable! Include x- and y-axis labels, a legend, etc.*

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to show the in- and out-degree distributions of nodes.</font>

<font size="+1" color="red">Replace this cell by a brief commentary, in your own words, about these degree distributions.</font>

<font size="+1" color="red">Replace this cell with code to generate a scatterplot of the in- and out-degrees of nodes in the LLC.</font>

<font size="+1" color="red">Replace this cell by a brief commentary, in your own words, about what you see on the scatterplot. What does it tell you about the largest strongly connected component of the graph?</font>

# 3. Create the undirected co-mention network


The **undirected co-mention network** connects two accounts if they are both mentioned in the same tweet. The weight of the edge is the number of tweets in which the accounts are co-mentioned.

Suppose the mentions in a Tweet are in the array ``mentions``, one way to iterate through all pairs of co-mentioned is like this:

```python
    for mention1 in mentions:
        for mention2 in mentions:
            ...
```

Read the original input file again to create a dictionary `co_mentions_counter` in which keys are tuples (user1, user2) in which user1 lexicographically precedes user2 (user1 < user2), and values are the number of times user1 and user2 have appeared together in a tweet.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to create the `co_mentions_counter`.</font>


Print the number of times the accounts `educaciocat` and `elnacionalcat` have been mentioned together. It should be 8.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>


In [ ]:
# Leave this code as-is

print(co_mentions_counter[('educaciocat', 'elnacionalcat')])

<font size="+1" color="red">Replace this cell with your code to print all pairs of accounts that have been co-mentioned 20 times or more.</font>

Now create a file named `OUTPUT_CO_MENTIONS_FILENAME` containing co-mentions in tab-separated columns `Source, Target, Weight`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to create the co-mentions file.</font>

# 4. Visualize the undirected co-mention network 

Read the co-mentions file and create a `Graph` in which nodes are accounts and the links between them are weighted by the number of co-mentions.

Find the nodes that have a **degree larger or equal to 17**, and then create a `subgraph` with those nodes and the edges between them.

Visualize the network using a `spring_layout` that uses the `weight` edge property. A few notes:

* You can take inspiration from the layout provided when we visualized the directed mentions network earlier in this practice. But try different versions — play around with the layout properties like `k`, the number of `iterations`, or the `gravity` (if using the 'energy' method).
* Try using the 'force' method of `spring_layout`, or a completely different layout if you'd like: check the NetworkX [Drawing](https://networkx.org/documentation/stable/reference/drawing.html) documentation for options.
* Keep in mind this network is larger, so it may take more time to visualize it. Usually the most time-consuming part is calculating the layout coordinates `pos`. Some layouts are faster than others.

Style the network as you wish, what matters in the end is that your visualization is readable and the layout and styling is meaningful.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to draw the network as just described.</font>

<font size="+1" color="red">Replace this cell by a brief commentary, in your own words, of what you see in this graph. Which kinds of connected components does it have? Are connected components sparse or dense? Is there any specially dense sub-graph within the largest connected component, what is it? Include any aspects that you find relevant.</font>

# DELIVER (individually)

Deliver a zip file containing:

* Your code as a Python notebook (a `.ipynb` file).
   * Remove all unnecessary elements
   * Add comments when needed
* Any png files that you inserted in the notebook

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, text, and figures were produced by myself.</font>